# Dynamic Bootstrapping

This notebook outlines some of my initial thoughts on using bootstrap samples of decreasing balance to learn models on highly imbalanced datasets.  The idea is to start training a model with bootstrap samples of balanced classes and gradually decay the balance to the true distribution as learning takes place much like the learning rate.  The hope is that this will allow the classifier to identify patterns related to the minority class and then gradually come to recognize the true distribution.  Hopefully it becomes less biased towards the minority as time goes on and the bootstrap samples approach the true distribution.

### Data

All data was taken from the https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction/data challenge.  Just unzip the Train.zip file into the directory with this notebook in order to run the following code.

In [1]:
import keras
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM
from keras.optimizers import SGD, Adam
import warnings
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, precision_recall_curve,average_precision_score
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
# Load in the data
data = pd.read_csv('../talks/BYU_2018/Train.csv', nrows=250000)
data.head()

,Id,Title,Body,Tags
0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning
3,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding
4,5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents


In [5]:
# Parameters
max_length = 350  # length of input sequences to the model
n_top_tags = 1  # n most prevelant tags to try to predict
vocab_size = 2000  # How many distinct tokens to take
char_model = False  # type of model to train (character or word)
batch_size = 128
num_epochs = 20

In [6]:
# Convert the tags and texts to lists for the keras tokenizer
tag_list = data['Tags'].tolist()
text_list = data['Body'].tolist()

print(tag_list[:25])
print("="*115)
print(text_list[:2])

['php image-processing file-upload upload mime-types', 'firefox', 'r matlab machine-learning', 'c# url encoding', 'php api file-get-contents', 'proxy active-directory jmeter', 'core-plot', 'c# asp.net windows-phone-7', '.net javascript code-generation', 'sql variables parameters procedure calls', '.net obfuscation reflector', 'algorithm language-agnostic random', 'postfix migration mdaemon', 'documentation latex3 expl3', 'windows-7', 'php url-routing conventions', 'r temporary-files', 'wpf binding', 'javascript code-generation playframework minify', 'php xml hash multidimensional-array simplexml-load-string', 'medical-science cancer healthcare', 'c# .net linq', 'actionscript-3 flex flex3', 'iis', 'c# linq string enumeration']
["<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter w

In [7]:
# Tokenize the labels
tag = 'php'
tag_matrix = np.array([[(tag in x.split(' ')) + 0.0] for x in tag_list])

In [8]:
# tag_tokenizer = Tokenizer(num_words=n_top_tags + 1)
# tag_tokenizer.fit_on_texts(tag_list)
# tag_matrix = tag_tokenizer.texts_to_matrix(tag_list)[:, 1:]

In [9]:
print(tag_matrix.shape)
print(tag_matrix)
# print(list(tag_tokenizer.word_index.keys())[:11])

(250000, 1)
[[1.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [12]:
print(np.sum(tag_matrix))
print(tag_matrix.shape)
print(np.sum(tag_matrix) /tag_matrix.shape[0] )

16161.0
(250000, 1)
0.064644


# Sequence Model

In [13]:
text_tokenizer = Tokenizer(num_words=vocab_size, char_level=char_model)
text_tokenizer.fit_on_texts(text_list)
text_matrix = text_tokenizer.texts_to_sequences(text_list)

# We have a numeric representation of the words in the questions
print(vocab_size)
print(text_matrix[:2])
print(list(text_tokenizer.word_index.keys())[:25])
print(list(text_tokenizer.word_index.keys())[vocab_size-25:vocab_size])

2000
[[1, 381, 51, 4, 335, 23, 34, 1868, 53, 9, 34, 142, 53, 92, 274, 334, 548, 1658, 44, 261, 53, 2, 99, 9, 16, 52, 45, 4, 795, 2, 166, 61, 607, 2, 79, 10, 798, 6, 44, 153, 28, 2, 79, 100, 1417, 61, 53, 79, 81, 795, 1, 1, 9, 56, 6, 86, 4, 335, 23, 2, 1868, 53, 9, 34, 142, 32, 1278, 2, 53, 1024, 45, 102, 1], [1, 12, 21, 1171, 3, 67, 536, 4, 583, 6, 844, 307, 136, 15, 903, 1694, 893, 4, 84, 16, 954, 9, 2, 746, 338, 22, 178, 240, 3, 311, 512, 43, 1028, 9, 2, 746, 338, 10, 1384, 536, 61, 954, 13, 9, 27, 59, 3, 62, 9, 56, 6, 86, 4, 774, 536, 32, 954, 1, 1, 1]]
['p', 'the', 'i', 'to', 'code', 'a', 'gt', 'lt', 'is', 'and', 'pre', 'in', 'this', 'of', 'it', 'that', 'for', '0', '1', 'have', 'my', 'on', 'if', 'with', 'but']
['directories', 'fragment', 'argv', 'nginx', 'uiimage', 'updating', 'identifier', 'v1', 'uk', 'relationship', 'annotation', 'eg', 'embedded', 'htaccess', "name'", 'conditions', 'putting', '0000', 'mysite', 'inf', 'fairly', 'went', 'emails', "'s", 'escape']


In [14]:
# Padd all sequences to the same size
X = sequence.pad_sequences(text_matrix, maxlen=max_length, padding='pre', truncating='post')

y = tag_matrix

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

In [15]:
# Try a sequence model instead
seq_model = Sequential()
seq_model.add(Embedding(vocab_size, 100, input_shape=(max_length, )))
seq_model.add(Dropout(.2))
seq_model.add(LSTM(64))
seq_model.add(Dropout(.2))
seq_model.add(Dense(n_top_tags, activation='sigmoid'))
seq_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
seq_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 350, 100)          200000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 350, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 242,305
Trainable params: 242,305
Non-trainable params: 0
_________________________________________________________________


In [17]:
normal_scores = []

In [18]:
for e in range(0, 5):
    seq_model.fit(x_train, y_train, epochs=1)
    predictions = seq_model.predict(x_val)
    f1_scores = f1_score(y_val, predictions  > 0.5)
    precision_scores = precision_score(y_val, predictions  > 0.5)
    recall_scores = recall_score(y_val, predictions  > 0.5)
    auc_scores = roc_auc_score(y_val, predictions)
    avg_precision_score = average_precision_score(y_val, predictions)
    normal_scores.append([e, f1_scores, precision_scores, recall_scores, auc_scores, avg_precision_score])
    print(f1_scores, precision_scores, recall_scores, auc_scores, avg_precision_score)

Epoch 1/1
200000/200000 [==============================] - 1628s 8ms/step - loss: 0.1607 - acc: 0.9445
0.6104846083511125 0.6077063106796117 0.6132884262094305 0.9036273054643326 0.5533845592324237
Epoch 1/1
200000/200000 [==============================] - 1775s 9ms/step - loss: 0.1290 - acc: 0.9524
0.6592004977445948 0.6699336073348088 0.6488058787507655 0.9468717575051999 0.6337338546582847
Epoch 1/1
200000/200000 [==============================] - 1637s 8ms/step - loss: 0.1189 - acc: 0.9554
0.6287519747235387 0.7367338543809132 0.5483772198407838 0.9353962757942824 0.6415367590008276
Epoch 1/1
200000/200000 [==============================] - 1535s 8ms/step - loss: 0.1106 - acc: 0.9594
0.6670047329276537 0.7445283018867924 0.6041028781383956 0.951222880383778 0.7084128775917947
Epoch 1/1
200000/200000 [==============================] - 1574s 8ms/step - loss: 0.1072 - acc: 0.9604
0.6803211535310503 0.7317589002467395 0.6356399265156154 0.9505345244447532 0.7087419667966223
Epoch 1/1
2

KeyboardInterrupt: 

In [19]:
predictions = seq_model.predict(x_val)
f1_scores = f1_score(y_val, predictions  > 0.5)
precision_scores = precision_score(y_val, predictions  > 0.5)
recall_scores = recall_score(y_val, predictions  > 0.5)
auc_scores = roc_auc_score(y_val, predictions)
average_precision_score(y_val, predictions)
print(f1_scores, precision_scores, recall_scores, auc_scores, )

0.6852538275584206 0.7233752977203131 0.6509491733006736 0.955704135463438


In [20]:
average_precision_score(y_val, predictions)

0.7067615662703475

In [21]:
normal_scores.append([e, f1_scores, precision_scores, recall_scores, auc_scores])

# Use Dynamic Bootstrapping 

Previously we just trained a model out of the box on the data.  Pefromance is fairly poor and most of the classes are just set to 0.  Here we use dynamic bootstrapping to initialize our model parameters and then train for a few epochs on the true dataset.

In [31]:
# Try a sequence model instead
seq_model_boot = Sequential()
seq_model_boot.add(Embedding(vocab_size, 100, input_shape=(max_length, )))
seq_model_boot.add(Dropout(.2))
seq_model_boot.add(LSTM(64))
seq_model_boot.add(Dropout(.2))
seq_model_boot.add(Dense(n_top_tags, activation='sigmoid'))
seq_model_boot.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
# Find the true ratio of the training set.  This is the lower bound on how imblanaced the classes
# are allowed to be
true_ratio = np.sum(y_train) / y_train.shape[0]

In [33]:
# So 313 bootstrap samples per epoch
# Each bootstrap sample has 512 examples in it
# We will use a batch size of 128

# true_ratio = imbalance_ratio * x ^ (steps_per_epoch  * num_epochs)
# true_ratio / imbalance_ratio = x ^ (steps_per_epoch  * num_epochs)
# log(true_ratio) - log(imbalance_ratio) = steps_per_epoch  *num_epochs log(x)
# log(true_ratio) - log(imbalance_ratio) / (steps_per_epoch * num_epochs) = log(x)
# 10 ^ (log(true_ratio) - log(imbalance_ratio) / (steps_per_epoch * num_epochs)) = x

def calculate_ratio_constant(num_data, batch_size, num_batches, num_epochs, true_ratio, imbalance_ratio):
    """Calculates the decay constant for the imbalance ratio"""
    steps_per_epoch = num_data / (batch_size * num_batches)
    return np.exp((np.log(true_ratio) - np.log(imbalance_ratio)) / (steps_per_epoch * num_epochs))

In [34]:
batch_size = 128
num_epochs = 10
num_batches_in_bootstrap_sample = 10
imbalance_ratio = .5
num_times_to_train_on_full_dataset = 3
decay_rate = calculate_ratio_constant(y.shape[0], batch_size, num_batches_in_bootstrap_sample, num_epochs, true_ratio, imbalance_ratio)
print(decay_rate)

0.9989518045012056


In [35]:
# Create a pandas dataframe of the x_train data to bootstrap sample
x_train_df = pd.DataFrame(x_train)
x_train_df['label'] = pd.Series(y_train.flatten())

In [36]:
# Create the positive and negative examples
x_train_pos_df = x_train_df[x_train_df['label'] == 1.0]
x_train_neg_df = x_train_df[x_train_df['label'] == 0.0]

In [37]:
# Bootstrap sample from the training set

# start balanced
sample_ration = .5

validation_performance = []

# Perform 4 epochs of training using bootstrap samples where the imbalance rate gradually approaches the true
# distribution
for i in range(0, num_epochs):
    for j in range(0, int(y_train.shape[0] / (num_batches_in_bootstrap_sample * batch_size))):
        pos_samples = (x_train_pos_df
                       .sample(int(num_batches_in_bootstrap_sample * batch_size * imbalance_ratio), replace=True)
                       .drop(columns='label')
                       .as_matrix())
        neg_samples = (x_train_neg_df
                       .sample(int(num_batches_in_bootstrap_sample * batch_size * (1 - imbalance_ratio)), replace=True)
                       .drop(columns='label')
                       .as_matrix())
        x_train_sampled = np.concatenate([pos_samples, neg_samples])
        y_train_pos = np.ones([pos_samples.shape[0], ])
        y_train_neg = np.zeros([neg_samples.shape[0], ])
        y_train_sampled = np.concatenate([y_train_pos, y_train_neg])
        seq_model_boot.fit(x_train_sampled, y_train_sampled, epochs=1, batch_size=batch_size, shuffle=True)
        
        # Update sample ratio
        imbalance_ratio = imbalance_ratio * decay_rate
    
    # Every epoch print and save the validation performance
    predictions = seq_model_boot.predict(x_val)
    f1_scores = f1_score(y_val, predictions  > 0.5)
    precision_scores = precision_score(y_val, predictions  > 0.5)
    recall_scores = recall_score(y_val, predictions  > 0.5)
    auc_scores = roc_auc_score(y_val, predictions)
    avg_precision_score = average_precision_score(y_val, predictions)
    validation_performance.append([f1_scores, precision_scores, recall_scores, auc_scores, avg_precision_score])
    print(f1_scores, precision_scores, recall_scores, auc_scores, avg_precision_score)

seq_model_boot.fit(x_train, y_train, epochs=num_times_to_train_on_full_dataset, shuffle=True)

Epoch 1/1
1280/1280 [==============================] - 5s 4ms/step - loss: 0.6922 - acc: 0.5375
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.6880 - acc: 0.5426
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.6536 - acc: 0.6388
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.5901 - acc: 0.7021
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.5958 - acc: 0.7217
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.5513 - acc: 0.7428
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.4581 - acc: 0.8030
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.4402 - acc: 0.8069
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.4020 - acc: 0.8405
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.3729 - acc: 0.8522
Epoch 1/1
1279/1279 [===================

1279/1279 [==============================] - 4s 3ms/step - loss: 0.3157 - acc: 0.8749
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2977 - acc: 0.8890
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.3176 - acc: 0.8772
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.3067 - acc: 0.8843
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.3136 - acc: 0.8788
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.2969 - acc: 0.8905
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.3041 - acc: 0.8905
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2925 - acc: 0.8890
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.3158 - acc: 0.8804
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.3289 - acc: 0.8757
Epoch 1/1
1279/1279 [=============================

1279/1279 [==============================] - 4s 3ms/step - loss: 0.2666 - acc: 0.8976
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2752 - acc: 0.8991
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2737 - acc: 0.8913
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2660 - acc: 0.9085
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2649 - acc: 0.8984
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2497 - acc: 0.9054
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2745 - acc: 0.8858
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2616 - acc: 0.8976
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2648 - acc: 0.8913
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2392 - acc: 0.8976
Epoch 1/1
1279/1279 [=============================

1279/1279 [==============================] - 4s 3ms/step - loss: 0.2355 - acc: 0.9085
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2240 - acc: 0.9171
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2076 - acc: 0.9148
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2512 - acc: 0.8984
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2745 - acc: 0.8929
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2294 - acc: 0.9046
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2386 - acc: 0.9132
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2423 - acc: 0.9085
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2423 - acc: 0.9070
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2278 - acc: 0.9234
Epoch 1/1
1279/1279 [=============================

1279/1279 [==============================] - 4s 3ms/step - loss: 0.2946 - acc: 0.8593
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.4064 - acc: 0.8038
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.2952 - acc: 0.8827
Epoch 1/1
1279/1279 [==============================] - 4s 4ms/step - loss: 0.2989 - acc: 0.8866
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2814 - acc: 0.8819
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2979 - acc: 0.8718
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2918 - acc: 0.8937
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2795 - acc: 0.8835
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.3171 - acc: 0.8702
Epoch 1/1
1279/1279 [==============================] - 4s 3ms/step - loss: 0.2865 - acc: 0.8890
Epoch 1/1
1279/1279 [=============================

...

1279/1279 [==============================] - 5s 4ms/step - loss: 0.1236 - acc: 0.9515
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1207 - acc: 0.9492
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1093 - acc: 0.9554
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1179 - acc: 0.9523
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1319 - acc: 0.9484
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1256 - acc: 0.9593
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1187 - acc: 0.9547
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.0979 - acc: 0.9640
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1100 - acc: 0.9601
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1216 - acc: 0.9507
Epoch 1/1
1279/1279 [=============================

1279/1279 [==============================] - 5s 4ms/step - loss: 0.1026 - acc: 0.9625
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1124 - acc: 0.9562
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1191 - acc: 0.9500
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.0964 - acc: 0.9640
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1113 - acc: 0.9586
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1517 - acc: 0.9461
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1164 - acc: 0.9523
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1122 - acc: 0.9578
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1111 - acc: 0.9578
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1247 - acc: 0.9492
Epoch 1/1
1279/1279 [=============================

1279/1279 [==============================] - 5s 4ms/step - loss: 0.1210 - acc: 0.9547
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1251 - acc: 0.9453
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.0952 - acc: 0.9633
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1041 - acc: 0.9586
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1177 - acc: 0.9507
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1108 - acc: 0.9539
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.0983 - acc: 0.9570
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1057 - acc: 0.9625
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1099 - acc: 0.9625
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1194 - acc: 0.9507
Epoch 1/1
1279/1279 [=============================

1279/1279 [==============================] - 5s 4ms/step - loss: 0.1029 - acc: 0.9586
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1040 - acc: 0.9586
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.0969 - acc: 0.9586
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1076 - acc: 0.9562
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.0914 - acc: 0.9687
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.0954 - acc: 0.9586
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1038 - acc: 0.9593
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.0849 - acc: 0.9617
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1016 - acc: 0.9562
Epoch 1/1
1279/1279 [==============================] - 5s 4ms/step - loss: 0.1025 - acc: 0.9601
Epoch 1/1
1279/1279 [=============================

KeyboardInterrupt: 

In [ ]:
# seq_model_boot.fit(x_train, y_train, epochs=num_times_to_train_on_full_dataset, shuffle=True)

In [39]:
predictions = seq_model_boot.predict(x_val)
f1_scores = f1_score(y_val, predictions  > 0.5)
precision_scores = precision_score(y_val, predictions  > 0.5)
recall_scores = recall_score(y_val, predictions  > 0.5)
auc_scores = roc_auc_score(y_val, predictions)
print(f1_scores, precision_scores, recall_scores, auc_scores)

0.68323793949305 0.7332397332397332 0.6396203306797306 0.9492628519380746


# Notes:

```
After 5 epochs trained on full data set we get:
0.5976387002403092
0.7844212835984641
0.48270042194092827

After 7
0.6183401154537712 0.7037914691943128 0.5513924050632911

After dynamic bootstrapping we get
0.6410598233627728
0.6330426197136745
0.6492827004219409
```

It appears that dynamic bootstrapping gives, better performance.  It also appears that it converges more quickly to the optimum value.

# Cost Sensitive Learning

Use class weighting and compare

# Over Sampleing
Here we just oversample the minority class to equal the majority class using bootstrap sampling.  We then assess the model performance. on 5 epochs.

In [40]:
# Try a sequence model instead
seq_model_over = Sequential()
seq_model_over.add(Embedding(vocab_size, 100, input_shape=(max_length, )))
seq_model_over.add(Dropout(.2))
seq_model_over.add(LSTM(64))
seq_model_over.add(Dropout(.2))
seq_model_over.add(Dense(n_top_tags, activation='sigmoid'))
seq_model_over.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [42]:
pos_samples = (x_train_pos_df
               .sample(x_train_neg_df.shape[0], replace=True)
               .drop(columns='label')
               .as_matrix())
neg_samples = (x_train_neg_df
               .drop(columns='label')
               .as_matrix())
x_train_sampled = np.concatenate([pos_samples, neg_samples])
y_train_pos = np.ones([pos_samples.shape[0], ])
y_train_neg = np.zeros([neg_samples.shape[0], ])
y_train_sampled = np.concatenate([y_train_pos, y_train_neg])

In [43]:
seq_model_over.fit(x_train_sampled, y_train_sampled, epochs=5, batch_size=batch_size, shuffle=True)

Epoch 1/5
374210/374210 [==============================] - 1380s 4ms/step - loss: 0.3137 - acc: 0.8700
Epoch 2/5
374210/374210 [==============================] - 1403s 4ms/step - loss: 0.2435 - acc: 0.9041
Epoch 3/5
374210/374210 [==============================] - 1437s 4ms/step - loss: 0.2344 - acc: 0.9092
Epoch 4/5
374210/374210 [==============================] - 1424s 4ms/step - loss: 0.1710 - acc: 0.9380
Epoch 5/5
374210/374210 [==============================] - 1421s 4ms/step - loss: 0.1404 - acc: 0.9518


In [44]:
predictions = seq_model_over.predict(x_val)
f1_scores = f1_score(y_val, predictions  > 0.5)
precision_scores = precision_score(y_val, predictions  > 0.5)
recall_scores = recall_score(y_val, predictions  > 0.5)
auc_scores = roc_auc_score(y_val, predictions)
print(f1_scores, precision_scores, recall_scores, auc_scores)

0.5750758606257195 0.4368144969003338 0.8413962033067973 0.9476964041988127


# Under Sampling

Here we understample the majority class to equal the minority class

In [45]:
# Try a sequence model instead
seq_model_under = Sequential()
seq_model_under.add(Embedding(vocab_size, 100, input_shape=(max_length, )))
seq_model_under.add(Dropout(.2))
seq_model_under.add(LSTM(64))
seq_model_under.add(Dropout(.2))
seq_model_under.add(Dense(n_top_tags, activation='sigmoid'))
seq_model_under.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
pos_samples = (x_train_pos_df
               .drop(columns='label')
               .as_matrix())
neg_samples = (x_train_neg_df
               .sample(x_train_pos_df.shape[0], replace=True)
               .drop(columns='label')
               .as_matrix())
x_train_sampled = np.concatenate([pos_samples, neg_samples])
y_train_pos = np.ones([pos_samples.shape[0], ])
y_train_neg = np.zeros([neg_samples.shape[0], ])
y_train_sampled = np.concatenate([y_train_pos, y_train_neg])

In [47]:
seq_model_under.fit(x_train_sampled, y_train_sampled, epochs=5, batch_size=batch_size, shuffle=True)

Epoch 1/5
25790/25790 [==============================] - 91s 4ms/step - loss: 0.5098 - acc: 0.7492
Epoch 2/5
25790/25790 [==============================] - 93s 4ms/step - loss: 0.3408 - acc: 0.8699
Epoch 3/5
25790/25790 [==============================] - 96s 4ms/step - loss: 0.3522 - acc: 0.8650
Epoch 4/5
25790/25790 [==============================] - 94s 4ms/step - loss: 0.4659 - acc: 0.8125
Epoch 5/5
25790/25790 [==============================] - 104s 4ms/step - loss: 0.3937 - acc: 0.8539


In [48]:
predictions = seq_model_under.predict(x_val)
f1_scores = f1_score(y_val, predictions  > 0.5)
precision_scores = precision_score(y_val, predictions  > 0.5)
recall_scores = recall_score(y_val, predictions  > 0.5)
auc_scores = roc_auc_score(y_val, predictions)
print(f1_scores, precision_scores, recall_scores, auc_scores)

0.3630207002300026 0.2293718714677862 0.8698714023270055 0.9098679249718364


# SMOTE

Use smote to create ideal dataset then run training for 10 epochs